In [1]:
import time
import  os
import jittor as jt
from jittor import nn
from jittor.models import resnet152
jt.flags.use_cuda = jt.has_cuda

batch_size = 8
warmup = 10
rerun = 1000
data = jt.random((batch_size, 3, 224, 224))
label= jt.randint(0,10,(batch_size,))
model = resnet152(num_classes=10)

loss_fn = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.parameters(),1e-3)

In [2]:

print('start warmup')
for i in range(warmup):
    pred = model(data)
    optimizer.step (loss_fn(pred, label))

print('start run')
start = time.time()
for i in range(rerun):
    print('\r',i,end='')
    pred = model(data)
    optimizer.step (loss_fn(pred, label))
    
end = time.time()

print('\n',end-start)
print("Jittor FPS:", (rerun*batch_size)/(end-start))


start warmup
start run
 999
 201.14806866645813
Jittor FPS: 39.77169680542907


In [1]:
import time
import torch
from torch import nn
from torchvision.models import resnet152

warmup = 10
rerun = 1000
batch_size = 8
data = torch.randn((batch_size, 3, 224, 224)).to('cuda:0')
label=torch.randint(0,10,(batch_size,)).to('cuda:0')
model = resnet152(num_classes=10)
model.to('cuda:0')
model.eval()


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 此段代码对pytorch进行热身，确保时间测试准确
print('start warmup')
torch.cuda.synchronize()
for i in range(warmup):
    optimizer.zero_grad()
    pred = model(data)
    loss_fn(pred,label).backward()
    optimizer.step()
# synchronize用于确保PyTorch计算完成
torch.cuda.synchronize()

# 开始测试运行时间
print('start run')
start = time.time()
for i in range(rerun):
    print('\r',i,end='')
    optimizer.zero_grad()
    pred = model(data)
    loss_fn(pred,label).backward()
    optimizer.step()
torch.cuda.synchronize()
end = time.time()

print('\n',end-start)
print("PyTorch FPS:", (rerun*batch_size)/(end-start))

D:\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


start warmup
start run
 999
 220.75602531433105
PyTorch FPS: 36.23910146329607


In [3]:
import numpy as np
batch_size = 8
y=np.random.randint(0,10,(batch_size,1))
import os
import tensorflow as tf
print(tf.__version__)

import tensorflow.keras.backend as K

tf.config.optimizer.set_jit(True)

warmup = 10
rerun = 2000
batch_size=8
data=K.random_normal((batch_size,224,224,3))
input_shape = data.shape[1:]
classes = 10
ds=tf.data.Dataset.from_tensor_slices((data,y)).repeat(-1).batch(8)

model=tf.keras.Sequential([
    tf.keras.applications.ResNet152(  
    weights=None, input_shape=input_shape, classes=classes),
    tf.keras.layers.Softmax()
])

model.compile(optimizer="adam", 
              loss="sparse_categorical_crossentropy"
             )

model.fit(ds,steps_per_epoch=10)

2.6.3
10/10 [==============================] - 67s 218ms/step - loss: 2.0259


In [4]:

model.fit(ds,steps_per_epoch=1000,verbose=1)

1000/1000 [==============================] - 221s 221ms/step - loss: 1.8362
